## Import libraries:

In [2]:
from imports import *

## 1. Preprocessing

The data corresponds to the Monte Carlo simulation of the SBND experiment. It was obtained by simulating the creation of neutrinos in the beam and their subsequent interaction in SBND, as well as the simulation of the detector's response to the particles resulting from the interaction of the neutrinos.

In [3]:
file_path = '/data/vidales/opana_tree_combined_v2609.root'
file = uproot.open(file_path)
optree = file['opanatree']['OpAnaTree'] # Tree con número de fotoelectrones
print("Keys in optree:", optree.keys())

Keys in optree: ['eventID', 'runID', 'subrunID', 'nuvX', 'nuvY', 'nuvZ', 'nuvT', 'nuvE', 'stepX', 'stepY', 'stepZ', 'stepT', 'dE', 'energydep', 'energydepX', 'energydepY', 'energydepZ', 'E', 'StartPx', 'StartPy', 'StartPz', 'EndPx', 'EndPy', 'EndPz', 'process', 'trackID', 'motherID', 'PDGcode', 'InTimeCosmics', 'InTimeCosmicsTime', 'dEtpc', 'dEpromx', 'dEpromy', 'dEpromz', 'dEspreadx', 'dEspready', 'dEspreadz', 'dElowedges', 'dEmaxedges', 'nopflash', 'flash_id', 'flash_time', 'flash_total_pe', 'flash_pe_v', 'flash_tpc', 'flash_y', 'flash_yerr', 'flash_z', 'flash_zerr', 'flash_x', 'flash_xerr', 'flash_ophit_time', 'flash_ophit_risetime', 'flash_ophit_starttime', 'flash_ophit_amp', 'flash_ophit_area', 'flash_ophit_width', 'flash_ophit_pe', 'flash_ophit_ch']


In [5]:
# Load variables to be predicted and associated data in a single step
hit_nuvT, hit_PE, hit_ch, hit_t, nopflash = (optree[key].array() for key in ['nuvT', 'flash_ophit_pe', 'flash_ophit_ch', 'flash_ophit_time','nopflash'])

# Filter out subarrays with more than one neutrino interaction
hit_nuvT_filtered, hit_PE_filtered, hit_ch_filtered, hit_t_filtered, nopflash_filtered = zip(*[
    (subarray_nuvT, subarray_PE, subarray_ch, subarray_t, subarray_nopflash) 
    for subarray_nuvT, subarray_PE, subarray_ch, subarray_t, subarray_nopflash in zip(hit_nuvT, hit_PE, hit_ch, hit_t, nopflash)
    if len(subarray_nuvT) ==1
])


"""
# If you want to upload only a small sample of the data, you can use the following code
total_entries = optree.num_entries
# Ex. Load the first 10% of the entries (you can change the range as needed)
entry_start, entry_stop = 0, int(total_entries * 0.1)
flash_nuvT_chunk = optree['nuvT'].array(entry_start=entry_start, entry_stop=entry_stop)


# Check for hit_nuvT all events with one neutrino only
all_length_one = all(ak.num(array, axis=0) == 1 for array in hit_nuvT_filtered)

# Output the result
if all_length_one:
    print("All arrays have length equal to one.")
else:
    print("Not all arrays have length equal to one.")

"""

'\n# If you want to upload only a small sample of the data, you can use the following code\ntotal_entries = optree.num_entries\n# Ex. Load the first 10% of the entries (you can change the range as needed)\nentry_start, entry_stop = 0, int(total_entries * 0.1)\nflash_nuvT_chunk = optree[\'nuvT\'].array(entry_start=entry_start, entry_stop=entry_stop)\n\n\n# Check for hit_nuvT all events with one neutrino only\nall_length_one = all(ak.num(array, axis=0) == 1 for array in hit_nuvT_filtered)\n\n# Output the result\nif all_length_one:\n    print("All arrays have length equal to one.")\nelse:\n    print("Not all arrays have length equal to one.")\n\n'

In [17]:
# Convert awkward arrays to Arrow tables
ophit_variables = pa.Table.from_pydict({
    'hit_nuvT_filtered': ak.to_list(hit_nuvT_filtered),
    'hit_PE_filtered': ak.to_list(hit_PE_filtered),
    'hit_ch_filtered': ak.to_list(hit_ch_filtered),
    'hit_t_filtered': ak.to_list(hit_t_filtered),
    'nopflash_filtered': ak.to_list(nopflash_filtered),
})

# Save to Parquet file
pq.write_table(ophit_variables, 'saved_archives/ophit_variables.parquet')

# To load some variables from the parquet file, use the following code
# table = pq.read_table('filtered_data.parquet')
# hit_nuvT_filtered = ak.from_iter(table.column('hit_nuvT_filtered').to_pylist())

## 2. PE and time matrix

In [ ]:
pe_matrix, time_matrix = process_photoelectrons(hit_PE_filtered, hit_ch_filtered, hit_t_filtered)

# Save the matrices to a CSV file
np.savetxt("saved_archives/pe_matrix_&_time_matrix/pe_matrix.csv", pe_matrix, delimiter=",")
np.savetxt("saved_archives/pe_matrix_&_time_matrix/time_matrix.csv", time_matrix, delimiter=",")